# Large Multi-Modal Foundation Model for Traffic Accident Analysis / Detection

In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import cv2

In [28]:
# Set constants for batch size and image dimensions
batch_size = 32
img_height = 224
img_width = 224

# Load and preprocess images
def preprocess_image(image):
    image = tf.image.resize(image, (250, 250))
    image = tf.cast(image, tf.float32) / 255.0
    return image



In [23]:
# Load training, testing, and validation datasets
train_data_dir = '/Users/manavgora/Desktop/Machine Learning/data/train'
test_data_dir = '/Users/manavgora/Desktop/Machine Learning/data/test'
val_data_dir = '/Users/manavgora/Desktop/Machine Learning/data/val'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    seed=101,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    seed=101,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    seed=101,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
)

Found 791 files belonging to 2 classes.
Found 100 files belonging to 2 classes.
Found 98 files belonging to 2 classes.


The provided code retrieves the class names from the training dataset (training_ds) and configures the datasets (training_ds and testing_ds) for performance using caching and prefetching techniques.

In [24]:
# Retrieve class names from training dataset
class_names = train_ds.class_names

# Configure dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Pre trained model - MobileNetV2

In [36]:
img_shape = (img_height, img_width, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False


9406464/9406464 [==============================] - 2s 0us/step



# Final Model Creation

In [37]:
model = tf.keras.Sequential([
    base_model,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(len(class_names), activation= 'softmax')
])


In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data = val_ds, epochs = 15)

In [ ]:
plt.plot(history.history['loss'], label = 'training loss')
plt.plot(history.history['accuracy'], label = 'training accuracy')
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(history.history['val_loss'], label = 'validation loss')
plt.plot(history.history['val_accuracy'], label = 'validation accuracy')
plt.grid(True)
plt.legend()